# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7fbda07df8b0>
├── 'a' --> tensor([[ 0.9775,  2.1718, -0.3166],
│                   [ 0.4725,  1.0343, -2.0935]])
└── 'x' --> <FastTreeValue 0x7fbda07dff10>
    └── 'c' --> tensor([[-0.4939,  1.8626,  0.4502, -0.3357],
                        [ 1.0979,  0.3314, -0.3358, -2.0419],
                        [ 1.5391, -0.4301,  0.7156, -1.0900]])

In [4]:
t.a

tensor([[ 0.9775,  2.1718, -0.3166],
        [ 0.4725,  1.0343, -2.0935]])

In [5]:
%timeit t.a

62.6 ns ± 0.0411 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7fbda07df8b0>
├── 'a' --> tensor([[-0.7596, -2.9364, -0.6133],
│                   [-1.1066, -1.2440, -0.8075]])
└── 'x' --> <FastTreeValue 0x7fbda07dff10>
    └── 'c' --> tensor([[-0.4939,  1.8626,  0.4502, -0.3357],
                        [ 1.0979,  0.3314, -0.3358, -2.0419],
                        [ 1.5391, -0.4301,  0.7156, -1.0900]])

In [7]:
%timeit t.a = new_value

60.2 ns ± 0.0449 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.9775,  2.1718, -0.3166],
               [ 0.4725,  1.0343, -2.0935]]),
    x: Batch(
           c: tensor([[-0.4939,  1.8626,  0.4502, -0.3357],
                      [ 1.0979,  0.3314, -0.3358, -2.0419],
                      [ 1.5391, -0.4301,  0.7156, -1.0900]]),
       ),
)

In [10]:
b.a

tensor([[ 0.9775,  2.1718, -0.3166],
        [ 0.4725,  1.0343, -2.0935]])

In [11]:
%timeit b.a

55.4 ns ± 0.0616 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.6320, -1.1212,  1.5461],
               [-0.2327,  0.1174, -0.4067]]),
    x: Batch(
           c: tensor([[-0.4939,  1.8626,  0.4502, -0.3357],
                      [ 1.0979,  0.3314, -0.3358, -2.0419],
                      [ 1.5391, -0.4301,  0.7156, -1.0900]]),
       ),
)

In [13]:
%timeit b.a = new_value

480 ns ± 0.0905 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

838 ns ± 0.954 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.3 µs ± 44.3 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

136 µs ± 666 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

137 µs ± 496 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7fbcf26d94f0>
├── 'a' --> tensor([[[ 0.9775,  2.1718, -0.3166],
│                    [ 0.4725,  1.0343, -2.0935]],
│           
│                   [[ 0.9775,  2.1718, -0.3166],
│                    [ 0.4725,  1.0343, -2.0935]],
│           
│                   [[ 0.9775,  2.1718, -0.3166],
│                    [ 0.4725,  1.0343, -2.0935]],
│           
│                   [[ 0.9775,  2.1718, -0.3166],
│                    [ 0.4725,  1.0343, -2.0935]],
│           
│                   [[ 0.9775,  2.1718, -0.3166],
│                    [ 0.4725,  1.0343, -2.0935]],
│           
│                   [[ 0.9775,  2.1718, -0.3166],
│                    [ 0.4725,  1.0343, -2.0935]],
│           
│                   [[ 0.9775,  2.1718, -0.3166],
│                    [ 0.4725,  1.0343, -2.0935]],
│           
│                   [[ 0.9775,  2.1718, -0.3166],
│                    [ 0.4725,  1.0343, -2.0935]]])
└── 'x' --> <FastTreeValue 0x7fbda07d64c0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

31.9 µs ± 63.7 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7fbceba69a60>
├── 'a' --> tensor([[ 0.9775,  2.1718, -0.3166],
│                   [ 0.4725,  1.0343, -2.0935],
│                   [ 0.9775,  2.1718, -0.3166],
│                   [ 0.4725,  1.0343, -2.0935],
│                   [ 0.9775,  2.1718, -0.3166],
│                   [ 0.4725,  1.0343, -2.0935],
│                   [ 0.9775,  2.1718, -0.3166],
│                   [ 0.4725,  1.0343, -2.0935],
│                   [ 0.9775,  2.1718, -0.3166],
│                   [ 0.4725,  1.0343, -2.0935],
│                   [ 0.9775,  2.1718, -0.3166],
│                   [ 0.4725,  1.0343, -2.0935],
│                   [ 0.9775,  2.1718, -0.3166],
│                   [ 0.4725,  1.0343, -2.0935],
│                   [ 0.9775,  2.1718, -0.3166],
│                   [ 0.4725,  1.0343, -2.0935]])
└── 'x' --> <FastTreeValue 0x7fbceba69a30>
    └── 'c' --> tensor([[-0.4939,  1.8626,  0.4502, -0.3357],
                        [ 1.0979,  0.3314, -0.3358, -2.0419],
                 

In [23]:
%timeit t_cat(trees)

30.2 µs ± 119 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

58.9 µs ± 449 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[-0.4939,  1.8626,  0.4502, -0.3357],
                       [ 1.0979,  0.3314, -0.3358, -2.0419],
                       [ 1.5391, -0.4301,  0.7156, -1.0900]],
              
                      [[-0.4939,  1.8626,  0.4502, -0.3357],
                       [ 1.0979,  0.3314, -0.3358, -2.0419],
                       [ 1.5391, -0.4301,  0.7156, -1.0900]],
              
                      [[-0.4939,  1.8626,  0.4502, -0.3357],
                       [ 1.0979,  0.3314, -0.3358, -2.0419],
                       [ 1.5391, -0.4301,  0.7156, -1.0900]],
              
                      [[-0.4939,  1.8626,  0.4502, -0.3357],
                       [ 1.0979,  0.3314, -0.3358, -2.0419],
                       [ 1.5391, -0.4301,  0.7156, -1.0900]],
              
                      [[-0.4939,  1.8626,  0.4502, -0.3357],
                       [ 1.0979,  0.3314, -0.3358, -2.0419],
                       [ 1.5391, -0.4301,  0.7156, -1.0900]],

In [26]:
%timeit Batch.stack(batches)

77.1 µs ± 173 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[-0.4939,  1.8626,  0.4502, -0.3357],
                      [ 1.0979,  0.3314, -0.3358, -2.0419],
                      [ 1.5391, -0.4301,  0.7156, -1.0900],
                      [-0.4939,  1.8626,  0.4502, -0.3357],
                      [ 1.0979,  0.3314, -0.3358, -2.0419],
                      [ 1.5391, -0.4301,  0.7156, -1.0900],
                      [-0.4939,  1.8626,  0.4502, -0.3357],
                      [ 1.0979,  0.3314, -0.3358, -2.0419],
                      [ 1.5391, -0.4301,  0.7156, -1.0900],
                      [-0.4939,  1.8626,  0.4502, -0.3357],
                      [ 1.0979,  0.3314, -0.3358, -2.0419],
                      [ 1.5391, -0.4301,  0.7156, -1.0900],
                      [-0.4939,  1.8626,  0.4502, -0.3357],
                      [ 1.0979,  0.3314, -0.3358, -2.0419],
                      [ 1.5391, -0.4301,  0.7156, -1.0900],
                      [-0.4939,  1.8626,  0.4502, -0.3357],
                   

In [28]:
%timeit Batch.cat(batches)

142 µs ± 506 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

303 µs ± 1.52 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
